In [10]:
import pandas as pd
import numpy as np
import os
import openpyxl

Transactions = pd.read_excel(
     os.path.join("99Bikers_Raw_data.xlsx"),
     engine='openpyxl',
     sheet_name='Transactions'
)
Transactions = Transactions.dropna(axis=1, how='all').dropna(axis=0, how='all')

NewCustomerList = pd.read_excel(
     os.path.join("99Bikers_Raw_data.xlsx"),
     engine='openpyxl',
     sheet_name='NewCustomerList'
)
NewCustomerList = NewCustomerList.dropna(axis=1, how='all').dropna(axis=0, how='all')

CustomerDemographic = pd.read_excel(
     os.path.join("99Bikers_Raw_data.xlsx"),
     engine='openpyxl',
     sheet_name='CustomerDemographic'
)
CustomerDemographic = CustomerDemographic.dropna(axis=1, how='all').dropna(axis=0, how='all')

CustomerAddress = pd.read_excel(
     os.path.join("99Bikers_Raw_data.xlsx"),
     engine='openpyxl',
     sheet_name='CustomerAddress'
)
CustomerAddress = CustomerAddress.dropna(axis=1, how='all').dropna(axis=0, how='all')

In [ ]:
Transactions['transaction_date'] = pd.to_datetime(Transactions['transaction_date'] , format='mixed')
Transactions['transaction_date']  = Transactions['transaction_date'].dt.strftime('%Y-%m-%d')
transaction_date = Transactions.transaction_date.unique()
transaction_date_data = [{'transaction_date_id': key + 1, "transaction_date": cnt} for key, cnt in enumerate(transaction_date)]
Transactions['transaction_date'].replace([i['transaction_date'] for i in transaction_date_data], [i['transaction_date_id'] for i in transaction_date_data], inplace=True)

Transactions = Transactions.fillna(2)
Transactions['online_order'] = Transactions['online_order'].astype(int)
Transactions['transaction_id'] = Transactions['transaction_id'].astype(int)
Transactions['product_id'] = Transactions['product_id'].astype(int)
Transactions['customer_id'] = Transactions['customer_id'].astype(int)

list_price = Transactions.list_price.unique()
list_price_data = [{'id': key + 1, "name": cnt} for key, cnt in enumerate(list_price)]

for list_item in list_price_data:
    Transactions.loc[Transactions['list_price'] == list_item['name'], 'product_id'] = list_item['id']

Product = Transactions[[
    'product_id', 
    'brand',
    'product_line',
    'product_class',
    'product_size'
    ]]

Product_unique = Product.drop_duplicates()
product_table_data = Product_unique.to_dict('records')

Customer = CustomerDemographic[[
    'customer_id',
    'first_name',
    'last_name',
    'gender',
    'past_3_years_bike_related_purchases',
    'DOB',
    'job_title',
    'job_industry_category',
    'wealth_segment',

    ]]
Customer_join = Customer.join(CustomerAddress.set_index('customer_id'), on='customer_id')
Customer_cleaned = Customer_join[[
    'customer_id',
    'first_name',
    'last_name',
    'gender',
    'past_3_years_bike_related_purchases',
    'DOB',
    'job_title',
    'job_industry_category',
    'wealth_segment',
    'address',
    'postcode',
    'state',
]]

Customer_cleaned.loc[Customer_cleaned['gender'] == 'F', 'gender'] = 'Female'
Customer_cleaned.loc[Customer_cleaned['gender'] == 'Femal', 'gender'] = 'Female'
Customer_cleaned.loc[Customer_cleaned['gender'] == 'M', 'gender'] = 'Male'
Customer_cleaned['DOB'] = pd.to_datetime(Customer_cleaned['DOB'] , format='mixed')
Customer_cleaned['DOB']  = Customer_cleaned['DOB'].dt.strftime('%Y-%m-%d')
Customer_cleaned = Customer_cleaned.fillna(0)
Customer_cleaned['customer_id'] = Customer_cleaned['customer_id'].astype(int)
Customer_cleaned['past_3_years_bike_related_purchases'] = Customer_cleaned['past_3_years_bike_related_purchases'].astype(int)
Customer_cleaned['postcode'] = Customer_cleaned['postcode'].astype(int)
customer_cleaned_data = Customer_cleaned.to_dict('records')

transactions_data = Transactions[[
    'transaction_id',
    'product_id',
    'customer_id',
    'transaction_date',
    'online_order',
    'list_price',
    'standard_cost',
]]
transactions_data = transactions_data.fillna(2)
transactions_data['online_order'] = transactions_data['online_order'].astype(int)
transactions_data = transactions_data.to_dict('records')


In [ ]:
from sqlalchemy import create_engine, schema 
from sqlalchemy_utils import database_exists, create_database

# Настройка соединения с БД
engine_string = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
    user = 'airflow',
    password = 'airflow',
    host = '192.168.1.57',
    port = 5432,
    database = 'bikers',
)

engine = create_engine(engine_string)
if not database_exists(engine.url):
    create_database(engine.url)



from sqlalchemy import MetaData, Table, Column, Integer, BigInteger, String, Date, Boolean, SmallInteger, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import inspect


def drop_table(table_name, engine=engine):
    Base = declarative_base()
    metadata = MetaData()
    metadata.reflect(bind=engine)
    table = metadata.tables[table_name]
    if table is not None:
        Base.metadata.drop_all(engine, [table], checkfirst=True)

inspector = inspect(engine)
for table_name in inspector.get_table_names():
    drop_table(table_name)




In [8]:

meta = MetaData()
transaction_date_table = Table(
   'transaction_date', meta,
   Column('transaction_date_id', SmallInteger, primary_key = True),
   Column('transaction_date', String),
)
meta.create_all(engine)

meta = MetaData()
product_table = Table(
   'product_table', meta,
   Column('product_id', Integer, primary_key = True),
   Column('brand', String),
   Column('product_line', String),
   Column('product_class', String),
   Column('product_size', String),
)
meta.create_all(engine)

meta = MetaData()
customer_table = Table(
   'customer_table', meta,
   Column('customer_id', Integer, primary_key = True),
   Column('first_name', String),
   Column('last_name', String),
   Column('past_3_years_bike_related_purchases', SmallInteger),
   Column('DOB', String),
   Column('job_title', String),
   Column('job_industry_category', String),
   Column('wealth_segment', String),
   Column('address', String),
   Column('postcode', SmallInteger),
   Column('state', String),
)
meta.create_all(engine)

meta = MetaData()
transactions_data_table = Table(
   'transactions_data_table', meta,
   Column('transaction_id', Integer, primary_key = True),
   Column('product_id', Integer),
   Column('customer_id', Integer),
   Column('transaction_date', String),
   Column('online_order', SmallInteger),
   Column('list_price', Float),
   Column('standard_cost', Float),

)
meta.create_all(engine)


In [9]:
from sqlalchemy import insert

with engine.connect() as conn:
  conn.execute(
    insert(transaction_date_table),
    transaction_date_data
  )

with engine.connect() as conn:
  conn.execute(
    insert(product_table),
    product_table_data
  )


with engine.connect() as conn:
  conn.execute(
    insert(customer_table),
    customer_cleaned_data
  )

with engine.connect() as conn:
  conn.execute(
    insert(transactions_data_table),
    transactions_data
  )

In [ ]:
Customer_join = Customer.join(CustomerAddress.set_index('customer_id'), on='customer_id')
Customer_join.head()
Transactions_join = Transactions.join(Customer_join.set_index('customer_id'), on='customer_id')
Transactions_join.head()

In [ ]:
Transactions_join.info()

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import mlflow
from sklearn.metrics import mean_squared_error, mean_absolute_error
from mlflow.models import infer_signature


In [26]:
mlflow.set_tracking_uri("localhost:8080")


In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(Transactions_join['list_price'], Transactions_join['past_3_years_bike_related_purchases'], train_size=0.8)

In [ ]:
model = LinearRegression()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
Y_pred_train = model.predict(X_train)

with mlflow.start_run():
    mlflow.log_metric('mse', mean_squared_error(Y_pred, Y_test))
    mlflow.log_metric('mae', mean_absolute_error(Y_pred, Y_test))

    mlflow.log_param('algorithm', 'Linear Regression')
    mlflow.log_param('intercept', model.intercept_)
    mlflow.log_param('coeff', model.coef_)

    fig, ax = plt.subplots(2)
    ax[0].scatter(X_test, Y_test)
    ax[0].scatter(X_test, Y_pred, 'b')

    ax[1].scatter(X_train, Y_train)
    ax[2].scatter(X_train, Y_pred_train, 'b')

    mlflow.log_figure(fig, 'results.png')

    signature = infer_signature(X_test, Y_test)

    mlflow.sklearn.log_model(model, 'sk_models', signature=signature)

    
